## 세그멘테이션(segmentation)이란
- 픽셀 수준에서 이미지의 각 부분이 어떤 의미를 갖는 영역인지 분리해 내는 방법


## 세그멘테이션의 종류

### 1) 시멘틱 세그멘테이션(semantic segmentation)
- 클래스 별 구분을 하나 클래스 안 객체들의 구분은 하지 않음

![image](https://user-images.githubusercontent.com/70703320/114801551-de1df700-9dd6-11eb-9326-be3971491814.png)

#### 시맨틱 세그멘테이션의 대표적인 모델인 U-Net

![image](https://user-images.githubusercontent.com/70703320/114801877-93e94580-9dd7-11eb-92d4-ebb9139a190b.png)

입력: 572x572 크기인 이미지 
출력: 388x388의 크기에 두 가지의 클래스를 가진 세그멘테이션 맵(segmentation map)

두 가지의 클래스를 문제에 따라 다르게 정의하면 클래스에 따른 시맨틱 세그멘테이션 맵(semantic segmentation map)을 얻을 수 있다.


### 2) 인스턴스 세그멘테이션(instance segmentation)
- 클래스 뿐 아니라 객체들에 대해서도 구분을 함

#### 인스턴스 세그멘테이션의 대표적인 모델인 Mask R-CNN
- 물체 검출(object detection) 모델로 각 개체를 구분하고 이후에 각 개체 별로 시맨틱 세그멘테이션을 수행

- MasK-R-CNN은 2-Stage Object Detection의 가장 대표적인 Faster-R-CNN을 계승한 것으로서, Faster-R-CNN의 아이디어인 RoI(Region-of-Interest) Pooling Layer(RoIPool) 개념을 개선하여 정확한 Segmentation에 유리하게 한 RoIAlign, 그리고 클래스별 마스크 분리라는 단순한 두가지 아이디어를 통해, 클래스별 Object Detection과 시멘틱 세그멘테이션을 사실상 하나의 Task로 엮어낸 것으로 평가받는 중요한 모델

- object 영역의 정확한 마스킹을 필요로 하는 Segmentation 문제에서 문제가 되는 것
    - RoIPool 과정에서 Quantization이 필요하다는 점
    
![image](https://user-images.githubusercontent.com/70703320/114803694-e4ae6d80-9dda-11eb-9a6d-32f1e0313bc0.png)




#### Q3. 위 그림에서, 가로 200, 세로 145의 RoI 영역에 대해 3X3의 RoIPool 을 적용했을 때, 정보를 잃어버리게 되는 영역의 색깔과, 불필요하게 포함되는 영역의 색깔은 각각 어떤 색인가요?

![스크린샷, 2021-04-15 11-28-24](https://user-images.githubusercontent.com/70703320/114805397-e6c5fb80-9ddd-11eb-9eca-b2a9034c7eab.png)
> 위의 512 X 512 이미지에서 오른쪽 중간의 고양이 영역의 크기는 200 X 145이다. 이를 VGG16에 적용하여 결과를 살펴보면 오른쪽 16 X 16 영역에서 빨간색 박스가 된다.


![스크린샷, 2021-04-15 11-28-37](https://user-images.githubusercontent.com/70703320/114805402-e7f72880-9ddd-11eb-9ee5-918401fb5af4.png)

> 하지만, 16 X 16 영역에 표시하려면 픽셀을 고려하여 주황색 영역만큼 차지하게 된다. 이제 이 이미지에 대해 3 X 3 RoI Pooling을 적용해보자. 이때, 주황색 영역의 크기는 6 X 4인데 가로 세로에 대해 3으로 나누주면 2, 1.33이 된다. 가로는 2개씩 적용할 것이고, 세로는 1.33이 애매하기 때문에 1로 적용할 것이다. 

> 이럴 경우 문제 위의 그림에서 초록색 부분은 불필요하게 추가되고, 연한 파랑색과 진한 파랑색은 손실이 일어나는 것이다.


3 X 3 Rolpool을 적용하면 손실되는 부분과 불필요하게 포함되는 영역이 생기는데 이를 방지하고자 보간법을 적용해준 방법이 RoIAlign방법이다.

#### RoIAlign

![image](https://user-images.githubusercontent.com/70703320/114806844-5dfc8f00-9de0-11eb-9a9d-85f70237d64f.png)

> 아이디어의 핵심은, RoI 영역을 pooling layer의 크기에 맞추어 등분한 후, RoIPool을 했을 때의 quantization 영역 중 가까운 것들과의 bilinear interpolation 계산을 통해 생성해야 할 Feature Map을 계산해 낸다는 점

> 빨간색 박스 그대로를 3 X 3으로 나누고 한 칸에 포함되는 부분적인 6개에서 4개를 골라 점을 찍는 방식으로 4번 진행한 것에서 특징정보를 뽑아내는 방법이다.


Mask R-CNN은 Faster R-CNN에서 특성 추출방식을 "RoIAlign" 방식으로 개선을 하고 세그멘테이션을 더한 방식

## 주요 세그멘테이션 모델 (1) FCN

- 기본적인 VGG 모델은 이미지의 특성을 추출하기 위한 네트워크의 뒷단에 fully connected layer를 붙여서 계산한 클래스별 확률을 바탕으로 이미지 분류를 수행

- FCN에서는 세그멘테이션을 하기 위해서 네트워크 뒷단에 fully connected layer 대신 CNN을 붙여줌

- **CNN을 붙여주는 이유는 FC layer의 경우 이미지의 위치 정보가 사라지고, 입력 이미지의 크기가 고정되기 때문**

- 그 결과 1x1의 커널 크기(kernel size)와 클래스의 개수만큼의 채널을 갖게 되고 클래스 히트맵을 얻을 수 있게 됨

- 클래스 히트맵은 원본 이미지보다 작게 되는데 (CNN, pooling layer를 거치기 때문) 이를 키워주는 방법이 upsampling이다. 

- upsampling에는 `deconvolution` 과 `interpolation` 방식이 존재한다.

- `deconvolution`는 컨볼루션 연산을 거꾸로 해준 것이다.

- `interpolation`은 보간법으로 주어진 값들을 통해 추정해야하는 픽셀들을 추정하는 방법이다.

- `interpolation`에는 linear와 bilinear 두 가지 방식이 존재한다.

    - 두 지점 사이의 값을 추정할 때 직관적으로 사용하는 방법이 Linear interpolation

    - 1차원의 Linear interpolation을 2차원으로 확장한 것이 bilinear interpolation


FCN-32s ::  FCN-32s, FCN-16s, FCN-8s로 결과를 구분해 설명합니다. FCN-16s는 앞쪽 블록에서 얻은 예측 결과 맵과, 2배로 upsampling한 맵을 더한 후, 한 번에 16배로 upsampling을 해주어 얻는다.






## 주요 세그멘테이션 모델 (2) U-Net

U-Net은 FCN에서 upsampling을 통해서 특성 맵을 키운 것을 입력값과 대칭적으로 만들어 준 것이다. 세포의 영역을 세그멘테이션 하기 위해 개발된 방법이다.

### 전체구조
---
좌측 :: Contracting path

![image](https://user-images.githubusercontent.com/70703320/114817664-64e1cc80-9df5-11eb-8997-e09721a78443.png)

우측 :: Expansive path

![image](https://user-images.githubusercontent.com/70703320/114817670-67442680-9df5-11eb-9a66-337f8f754334.png)

마지막 세그멘테이션 맵의 크기가 입력 이미지와 다른 것은 앞에서 이야기한 것처럼 세그멘테이션 맵을 원하는 크기로 조정하여(resize) 해결할 수 있다.


### 타일 기법
---
![image](https://user-images.githubusercontent.com/70703320/114818215-59db6c00-9df6-11eb-8abd-fe92c6f0d38a.png)

- U-Net이 downsampling과 upsampling(또는 인코딩과 디코딩)을 대칭으로 하는 점은 구조에서 한눈에 파악이 되고 얻을 수 있는 세그멘테이션 맵의 해상도의 차이도 존재한다.

- FCN은 입력 이미지의 크기를 조정하여 세그멘테이션 맵을 얻어냈으나 U-Net은 타일(tile) 방식을 사용해서 어느 정도 서로 겹치는 구간으로 타일을 나누어 네트워크를 추론, 큰 이미지에서도 높은 해상도의 세그멘테이션 맵을 얻을 수 있도록 하였다. 


### 데이터 불균형 해결
---
![image](https://user-images.githubusercontent.com/70703320/114818223-5e078980-9df6-11eb-8f8d-8bcbb524963c.png)


- 데이터셋에 세포나 배경보다는 절대적으로 세포 간 경계의 면적이 작을 것이기에 클래스 간 데이터 양의 불균형을 해결해 주기 위해서 분포를 고려한 weight map을 학습 때 사용했다. 여기서 weight는 손실함수(loss)에 적용되는 가중치이다.

- 의료 영상에서 세포 내부나 배경보다는 상대적으로 면적이 작은 세포 경계를 명확하게 추론해 내는 것이 더욱 중요하기 때문에, 세포 경계의 손실에 더 많은 페널티를 부과하는 방식으로 진행한다.

## 주요 세그멘테이션 모델 (3) DeepLab 계열


### 전체구조
---
![image](https://user-images.githubusercontent.com/70703320/114819396-62cd3d00-9df8-11eb-921a-405022cb9cca.png)

- U-Net에서의 Contracting path과 Expansive path의 역할을 하는 것이 여기서는 위 그림의 `인코더(Encoder)`, `디코더(Decoder)`이다.

- `인코더` :: 이미지에서 필요한 정보를 특성으로 추출해내는 모듈

- `디코더` :: 추출된 특성을 이용해 원하는 정보를 예측하는 모듈


### Atrous Convolution
---

![image](https://user-images.githubusercontent.com/70703320/114820016-4ed60b00-9df9-11eb-8faf-aaf17f236296.png)

- 띄엄띄엄 보는 컨볼루션
- 우측의 Atrous Convolution은 좌측의 일반적인 컨볼루션과 달리 더 넓은 영역을 보도록 해주기 위한 방법으로 커널이 일정 간격으로 떨어져 있다.
- 컨볼루션 레이어를 너무 깊게 쌓지 않아도 넓은 영역의 정보를 커버할 수 있게 된다.


### Spatial Pyramid Pooling
---

![image](https://user-images.githubusercontent.com/70703320/114820027-52699200-9df9-11eb-97d8-8dcfbd3ce7cb.png)

-  여러 가지 스케일로 convolution과 pooling을 하고 나온 다양한 특성을 연결(concatenate)해준다. 이를 통해서 멀티스케일로 특성을 추출하는 것을 병렬로 수행하는 효과를 얻을 수 있다.

-  여기서 컨볼루션을 Atrous Convolution으로 바꾸어 적용한 것은 Atrous Spatial Pyramid Pooling이다.


## 세그멘테이션의 평가

시맨틱 세그멘테이션의 결괏값은 이미지의 크기에 맞는 세그멘테이션 맵 크기 와 시맨틱 클래스의 수에 맞는 채널 크기를 갖는다.


### 픽셀별 정확도
---
![image](https://user-images.githubusercontent.com/70703320/114821656-d9b80500-9dfb-11eb-80eb-57cb29536b6b.png)

Pixel Accuracy는 쉽게 말해서 픽셀에 따른 정확도를 의미한다. 픽셀 별 이미지 분류 문제로 평가하는 경우에는 픽셀 별로 정답 클래스를 맞추었는지 여부, 즉 True/False를 구분한다.

### 마스크 IoU
---
물체 검출 모델을 평가할 때는 정답 라벨(ground truth)와 예측 결과 바운딩 박스(prediction bounding box) 사이의 IoU(intersection over union)를 사용한다.

마스크 IoU를 클래스 별로 계산하면 한 이미지에서 여러 클래스에 대한 IoU 점수를 얻을 수 있고 이를 평균하면 전체적인 시맨틱 세그멘테이션 성능을 가늠할 수 있다.



## Upsampling의 다양한 방법

Convolution layer와 다양한 Pooling 등으로 Feature의 크기를 줄여왔는데, 반대로 키우는 방법들에 대해 알아보자.

### Nearest Neighbor
---
![image](https://user-images.githubusercontent.com/70703320/114822340-dd985700-9dfc-11eb-8324-dabbd5b635de.png)

- Nearest upsampling은 scale을 키운 위치에서 원본에서 가장 가까운 값을 그대로 적용하는 방법이다.

### Bilinear Interpolation
---
![image](https://user-images.githubusercontent.com/70703320/114822349-e1c47480-9dfc-11eb-94d1-de1a6af5d6e3.png)

- Bilinear Interpolation은 두 축에 대해서 선형보간법을 통해 필요한 값을 메우는 방식이다. 

> $R_1$이 $Q_{11}$과 $Q_{21}$의 x축방향의 interpolation 결과
> $R_2$ 는 $Q_{11}$과 $Q_{21}$의 x축방향의 interpolation 결과
> $R_1$과 $R_2$를 y축방향으로 interpolation하면 새로운 위치 $P$의 값을 추정할 수 있다.

### Transposed Convolution
--- 
![image](https://user-images.githubusercontent.com/70703320/114822363-e5f09200-9dfc-11eb-811f-5804cdb0cd63.png)

- 지금까지 봤던 방식은 정해진 규칙에 따라서 upscaling하는 방법이었다. 학습할 수 있는 파라미터를 가진 Upsampling 방법 중 하나가 Transposed Convolution이다.

- 우리가 사용해왔던 Convolution Layer는 Kernel의 크기를 정의하고 입력된 Feature를 Window에 따라서 output을 계산하는데 Transposed Convolution이 이와 반대의 연산을 한다. 따라서 거꾸로 학습된 파라미터로 입력된 벡터를 통해 더 넓은 영역의 값을 추정해낸다.
